# TempoQL Demo

Before running, please `cd` into this repo and run `pip install .` in the virtual environment of your choice. This will install required dependencies so that you can run this notebook.

This example will use the MIMIC-IV-OMOP dataset available [here](https://physionet.org/content/mimic-iv-demo-omop/0.9/). Please download it and set the path to it in the `base_path` variable below.

In [ ]:
from tempo_ql import GenericDataset, formats, QueryEngine, FileVariableStore
import duckdb
import numpy as np
from pathlib import Path
import pandas as pd

In [ ]:
base_path = 'mimic-iv-demo-data-in-the-omop-common-data-model-0.9/1_omop_data_csv'

# provide your Gemini API key here if you'd like to try the LLM-assisted authoring workflow
gemini_api_key = None

In [ ]:
# Add data from the MIMIC-IV OMOP dataset

local_db = duckdb.connect(":memory:myconn")
for csv_file in Path(base_path).glob('*.csv'):
    table_name = csv_file.stem
    local_db.execute(f"create table {table_name} as select * from read_csv_auto('{csv_file}', header=true, ignore_errors=true, parallel=false)")

# Initialize query engine and variable store
var_store = FileVariableStore('example_data')
query_engine = QueryEngine(GenericDataset("duckdb:///:memory:myconn", formats.omop(id_field='person_id', use_source_concept_ids=True, concept_id_field='concept_id'), verbose=True), 
                           variable_stores=[var_store])

In [ ]:
query_engine.dataset.get_scopes()

In [ ]:
# get available concepts
names = query_engine.dataset.list_data_elements(scope='Procedure', return_counts=True)
names.head(20)

In [ ]:
# Get all visit occurrence IDs
query_engine.get_ids()

## Interactive Python Widget

Looking for inspiration? Try out the following queries. (Note that the MIMIC-IV OMOP sample has some format inconsistencies so some concepts that are available in the traditional MIMIC format may not be queryable.)

**Basic**
* How long are the ICU stays? `duration({Visit})`
* Which concepts are used to label sodium chloride infusions? `{name contains /sodium chloride/i; scope = Drug}`

**Aggregations**
* How often do patients have AFib recorded? `exists {AF (Atrial Fibrillation); scope = Condition} from #now - 24 hours to #now every 24 hours`
* How often are there recent temperature measurements? `count {Temperature Celsius; scope = Measurement} from #now - 4 hours to #now every 4 hours`
* What's the patient's global minimum mean blood pressure? `min {Non Invasive Blood Pressure mean; scope = Measurement} from #mintime to #maxtime`

In [ ]:
# Start an interactive widget, using the LLM assistant if API key is provided
query_engine.interactive(api_key=gemini_api_key)

In [ ]:
# Work in a JSON file to save queries (e.g. for version control)
query_engine.interactive(file_path='example_data/test_queries.json', api_key=gemini_api_key)

In [ ]:
# Later, query from your saved file
query_engine.query_from('example_data/test_queries.json')

## Running Queries in Code

In [ ]:
# perform one-off queries
visits = query_engine.query("{name contains /insulin/i; scope = Drug}")
visits

In [ ]:
# See what SQL queries were run after your last query_engine.query() call
print(query_engine.dataset._captured_queries)

In [ ]:
# Run future queries only within a random subset if desired
random_sample = np.random.choice(query_engine.get_ids(), size=50)
query_engine.dataset.set_trajectory_ids(random_sample)

In [ ]:
# we can store variables by name...
var_store['SimpleTemperature'] = query_engine.query("union({Temperature Celsius}, ({Temperature Fahrenheit} - 32) * 5 / 9) where #value < 50")

In [ ]:
# ... and then use their results later
query_engine.query("last SimpleTemperature before #now every 24 hours", return_subqueries=True)